In [ ]:
def perform_basic_tasks():
  import langchain
  import openai

  from sec import OpenAIkey
  key = OpenAIkey

  from langchain.chat_models import AzureChatOpenAI
  from langchain.prompts import PromptTemplate
  from langchain.chains import LLMChain
  global llm
  llm = AzureChatOpenAI(azure_endpoint= 'https://amazebotopenai.openai.azure.com/', api_key = key, deployment_name= 'Chat', api_version='2023-05-15')
perform_basic_tasks()

In [ ]:
output = llm.predict('What is the capital of England?')
print(output)

London


In [ ]:
prompt = PromptTemplate.from_template("What is the capital of {place}?")
chain1 = LLMChain(llm=llm, prompt=prompt)

prompt = PromptTemplate.from_template("What are the famous food of {capital}")
chain2 = LLMChain(llm=llm, prompt=prompt)

In [ ]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [chain1, chain2])
output = chain.run('India')
print(output)

The famous food of New Delhi includes:

1. Butter Chicken
2. Chole Bhature
3. Parathas
4. Kebabs
5. Biryani
6. Chaat
7. Samosas
8. Dosa
9. Golgappas
10. Kulfi Faluda
11. Rabri Falooda
12. Lassi
13. Aloo Tikki
14. Rajma Chawal
15. Nihari.


In [ ]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
synopsis_template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
synopsis_prompt_template = PromptTemplate(
    input_variables=["title", "era"], template=synopsis_template
)
synopsis_chain = LLMChain(
    llm=llm, prompt=synopsis_prompt_template, output_key="synopsis"
)

# This is an LLMChain to write a review of a play given a synopsis.
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[synopsis_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True,
)

output = overall_chain({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})
print(output)

In [ ]:
llm.predict("What is 13 raised to the .3432 power ?")

'Using a calculator, 13 raised to the power of 0.3432 is approximately 2.272.'

**Agent tools to solve complex math and science questions**
https://vteam.ai/blog/posts/mathematics-using-ll-ms-and-langchain<br>
https://github.com/gkamradt/langchain-tutorials/blob/main/agents/Agents.ipynb<br>
**How to provide knowledge base to the agent(RAG, Retrieval Question Answering langchain)**
1. https://www.youtube.com/watch?v=1towAoXOWLg
2. https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed
3. https://www.youtube.com/watch?app=desktop&v=iGZ0cV-SRLI
4. https://medium.com/@plbiojout/knowledge-based-agent-deployment-with-langchain-pinecone-and-phospho-bf8dffd1026d
5. https://js.langchain.com/docs/use_cases/autonomous_agents/sales_gpt
6. https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents
<br>

In [ ]:
from langchain import LLMMathChain
llm_math = LLMMathChain.from_llm(llm, verbose=True)
llm_math.run("What is 13 raised to the .3432 power ?")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power ?```text
13 ** 0.3432
```
...numexpr.evaluate("13 ** 0.3432")...

Answer: 2.4116004626599237
> Finished chain.


'Answer: 2.4116004626599237'

In [ ]:
from langchain_experimental.llm_symbolic_math.base import LLMSymbolicMathChain
llm_symbolic_math = LLMSymbolicMathChain.from_llm(llm, verbose=True)
llm_symbolic_math.run("What is the derivative of sin(x)*exp(x) with respect to x ?")



> Entering new LLMSymbolicMathChain chain...
What is the derivative of sin(x)*exp(x) with respect to x ?```text
diff(sin(x)*exp(x), x)
```
...sympy.sympify("diff(sin(x)*exp(x), x)")...

Answer: exp(x)*sin(x) + exp(x)*cos(x)
> Finished chain.


'Answer: exp(x)*sin(x) + exp(x)*cos(x)'

In [ ]:
# llm_symbolic_math.run("x = y + 5, y = z - 3,  z = x * y. Solve for x, y, z.")

In [ ]:
from langchain_experimental.pal_chain import PALChain
pal_chain = PALChain.from_math_prompt(llm=llm, verbose=True)
pal_chain.run("There is a group of 10 people who are ordering pizza. If each person gets 2 slices and each pizza has 4 slices, how many pizzas should they order?")



> Entering new PALChain chain...
def solution():
    """There is a group of 10 people who are ordering pizza. If each person gets 2 slices and each pizza has 4 slices, how many pizzas should they order?"""
    num_people = 10
    slices_per_person = 2
    slices_per_pizza = 4
    total_slices_needed = num_people * slices_per_person
    pizzas_needed = total_slices_needed // slices_per_pizza + (1 if total_slices_needed % slices_per_pizza != 0 else 0)
    result = pizzas_needed
    return result

> Finished chain.


'5'

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()

python_repl = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run, handle_parsing_errors=True )
# handle_parsing_errors=True, is important. Without this it doesn't give correct answer

pal_math = Tool(
    name="pal_math",
    description="A LLM Chain which helps to solve word problems.",
    func=pal_chain.run, handle_parsing_errors=True
  )

# tools = load_tools(['llm-math', 'python_repl', 'wolfram-alpha', 'pal-math'], llm=llm)
tools = load_tools(['llm-math'], llm=llm)
tools.append(python_repl)
tools.append(pal_math)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# agent.run("x = y + 5, y = z - 3,  z = x * y. Solve for x, y, z.")
# agent.run("x = y + 5, y = z - 3,  z = 5. Solve for x, y, z.") # Ek baar mein sahi answer nahi deta, 3rd time run kar ke sahi answer diya.
# agent.run("There is a group of 10 people who are ordering pizza. If each person gets 2 slices and each pizza has 4 slices, how many pizzas should they order?")

In [ ]:
from sec import WolframAlphaKey
import os
os.environ["WOLFRAM_ALPHA_APPID"] = WolframAlphaKey

from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain_experimental.utilities import PythonREPL

tools = load_tools(['llm-math', 'wolfram-alpha'], llm=llm)
# tools.append(python_repl)
# tools.append(pal_math)
agent = initialize_agent(tools, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("x = y + 5, y = z - 3,  z = x * y. Solve for x, y, z.") # Only Wolfram Alpha is able to give correct result for this question.



> Entering new AgentExecutor chain...
This is a system of equations, so we need to use substitution or elimination to solve for the variables.
Action: wolfram_alpha
Action Input: "solve x = y + 5, y = z - 3, z = x * y"
Observation: Assumption: solve x = y + 5
y = z - 3
z = x y 
Answer: x = 3 - sqrt(7) ∧ y = -2 - sqrt(7) ∧ z = 1 - sqrt(7)
Thought:We have the solution for x, y, and z.
Final Answer: x = 3 - sqrt(7), y = -2 - sqrt(7), z = 1 - sqrt(7)

> Finished chain.


'x = 3 - sqrt(7), y = -2 - sqrt(7), z = 1 - sqrt(7)'

In [ ]:
import os
os.environ["WOLFRAM_ALPHA_APPID"] = WolframAlphaKey

from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain_experimental.utilities import PythonREPL

tools = load_tools(['llm-math', 'wolfram-alpha'], llm=llm)
# tools.append(python_repl)
tools.append(pal_math)
agent = initialize_agent(tools, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("x = y + 5, y = z - 3,  z = x * y. Solve for x, y, z.") # Only Wolfram Alpha is able to give correct result for this question.



> Entering new AgentExecutor chain...
This is a system of equations that can be solved using substitution or elimination.
Action: pal_math
Action Input: "Solve x = y + 5, y = z - 3, z = x * y"

> Entering new PALChain chain...
def solution():
    """Solve x = y + 5, y = z - 3, z = x * y"""
    z = None  # we don't know the value of z yet
    y = z + 3  # from y = z - 3
    x = y + 5  # from x = y + 5
    z = x * y  # from z = x * y
    result = (x, y, z)
    return result

> Finished chain.

Observation: TypeError("unsupported operand type(s) for +: 'NoneType' and 'int'")
Thought:I need to input the equations in a different format. 
Action: pal_math
Action Input: "Solve the system of equations: x=y+5, y=z-3, z=x*y"

> Entering new PALChain chain...
def solution():
    """Solve the system of equations: x=y+5, y=z-3, z=x*y"""
    # Rearrange the first equation to get y in terms of x
    # y = x - 5
    # Substitute this into the second equation to get z in terms of x
    # z = x*(x-5) 

'x = 6.45, y = 1.45, z = 4.45.'

In [ ]:
import os
os.environ["WOLFRAM_ALPHA_APPID"] = WolframAlphaKey

from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain_experimental.utilities import PythonREPL

tools = load_tools(['llm-math', 'wolfram-alpha'], llm=llm)
tools.append(python_repl)
tools.append(pal_math)
agent = initialize_agent(tools, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)

# Wolfram Alpha is not good for science questions
agent.run("Eight copper wire of length L and diameter D are joined in parallel to form a single composite conductor of resistance R. If a single copper wire of length 2L have the same resistance R then its diameter will be")
# Unknown format
# agent.run("The total number of functions, f : {1, 2, 3, 4} → {1, 2, 3, 4, 5, 6} such that f(1) + f(2) = f(3), is equal to ?")
# Iteration depth or limit reached
# agent.run("One end of a horizontal thick copper wire of length 2L and radius 2R is welded to an end of another horizontal thin copper wire of length L and radius R. When the arrangement is stretched by applying forces at two ends, the ratio of the elongation in the thin wire to that in the thick wire is")

In [ ]:
# from langchain.chains import RetNWrite

# Loading the NCERT Textbook PDFs

### Loading Chemistry Textbook to Vector DB

In [ ]:
# https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed ( RAG )
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("11Chemistry/Chapter 1.pdf"),
    PyPDFLoader("11Chemistry/Chapter 2.pdf"),
]


docs = []
for loader in loaders:
    docs.extend(loader.load()) # loads each page as a list
# print(len(docs)) # len = total no. of pages

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

docs = text_splitter.split_documents(docs)
print(len(docs))

1320


In [ ]:
# # Convert Document object to text form. This preprocessing is needed to give it as an input to MiniLM embedding
# docs_text = []

# for doc in docs:
#     text = doc.page_content
#     docs_text.append(text)
# print(len(docs_text))

191


# MiniLM Embedding

In [ ]:
# from sentence_transformers import SentenceTransformer
# sentences = ["This is an example sentence", "Each sentence is converted"]

# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# embeddings = model.encode(sentences)
# print(len(embeddings[0]))
# print(embeddings[0].shape)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# doc_result = embedding.embed_documents(docs_text)
# doc_result = embedding.embed_texts(docs_text)
# print(len(doc_result[0]))

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Storing the embeddings in Vector DB

In [ ]:
from langchain.vectorstores import Chroma

# We save this vector store in a persistent directory so that we can use it in future.
persist_directory = 'docs/chemistry/chroma/'

# Create the vector store
chemistryVectorDB = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

print(chemistryVectorDB._collection.count())

1320


### Loading Chemistry, Math and Physics NCERT Textbook to Vector DB

In [ ]:
from langchain.document_loaders import PyPDFLoader
loaders = [
    PyPDFLoader("11Chemistry/Chapter 1.pdf"),
    PyPDFLoader("11Chemistry/Chapter 2.pdf"),
    PyPDFLoader("11Chemistry/Chapter 3.pdf"),
    PyPDFLoader("11Chemistry/Chapter 4.pdf"),
    PyPDFLoader("11Chemistry/Chapter 5.pdf"),
    PyPDFLoader("11Chemistry/Chapter 6.pdf"),
    PyPDFLoader("11Chemistry/Chapter 7.pdf"),
    PyPDFLoader("11Chemistry/Chapter 8.pdf"),
    PyPDFLoader("11Chemistry/Chapter 9.pdf"),
    PyPDFLoader("11Chemistry/Answers.pdf"),

    PyPDFLoader("12Chemistry/Chapter 1.pdf"),
    PyPDFLoader("12Chemistry/Chapter 2.pdf"),
    PyPDFLoader("12Chemistry/Chapter 3.pdf"),
    PyPDFLoader("12Chemistry/Chapter 4.pdf"),
    PyPDFLoader("12Chemistry/Chapter 5.pdf"),
    PyPDFLoader("12Chemistry/Chapter 6.pdf"),
    PyPDFLoader("12Chemistry/Chapter 7.pdf"),
    PyPDFLoader("12Chemistry/Chapter 8.pdf"),
    PyPDFLoader("12Chemistry/Chapter 9.pdf"),
    PyPDFLoader("12Chemistry/Chapter 10.pdf"),
    PyPDFLoader("12Chemistry/Answer.pdf"),

    PyPDFLoader("11Maths/Ch 1.pdf"),
    PyPDFLoader("11Maths/Ch 2.pdf"),
    PyPDFLoader("11Maths/Ch 3.pdf"),
    PyPDFLoader("11Maths/Ch 4.pdf"),
    PyPDFLoader("11Maths/Ch 5.pdf"),
    PyPDFLoader("11Maths/Ch 6.pdf"),
    PyPDFLoader("11Maths/Ch 7.pdf"),
    PyPDFLoader("11Maths/Ch 8.pdf"),
    PyPDFLoader("11Maths/Ch 9.pdf"),
    PyPDFLoader("11Maths/Ch 10.pdf"),
    PyPDFLoader("11Maths/Ch 11.pdf"),
    PyPDFLoader("11Maths/Ch 12.pdf"),
    PyPDFLoader("11Maths/Ch 13.pdf"),
    PyPDFLoader("11Maths/Ch 14.pdf"),
    PyPDFLoader("11Maths/Answers.pdf"),

    PyPDFLoader("12Maths/Ch 1.pdf"),
    PyPDFLoader("12Maths/Ch 2.pdf"),
    PyPDFLoader("12Maths/Ch 3.pdf"),
    PyPDFLoader("12Maths/Ch 4.pdf"),
    PyPDFLoader("12Maths/Ch 5.pdf"),
    PyPDFLoader("12Maths/Ch 6.pdf"),
    PyPDFLoader("12Maths/Ch 7.pdf"),
    PyPDFLoader("12Maths/Ch 8.pdf"),
    PyPDFLoader("12Maths/Ch 9.pdf"),
    PyPDFLoader("12Maths/Ch 10.pdf"),
    PyPDFLoader("12Maths/Ch 11.pdf"),
    PyPDFLoader("12Maths/Ch 12.pdf"),
    PyPDFLoader("12Maths/Ch 13.pdf"),
    PyPDFLoader("12Maths/Answers.pdf"),

    PyPDFLoader("11Physics/Chapter 1.pdf"),
    PyPDFLoader("11Physics/Chapter 2.pdf"),
    PyPDFLoader("11Physics/Chapter 3.pdf"),
    PyPDFLoader("11Physics/Chapter 4.pdf"),
    PyPDFLoader("11Physics/Chapter 5.pdf"),
    PyPDFLoader("11Physics/Chapter 6.pdf"),
    PyPDFLoader("11Physics/Chapter 7.pdf"),
    PyPDFLoader("11Physics/Chapter 8.pdf"),
    PyPDFLoader("11Physics/Chapter 9.pdf"),
    PyPDFLoader("11Physics/Chapter 10.pdf"),
    PyPDFLoader("11Physics/Chapter 11.pdf"),
    PyPDFLoader("11Physics/Chapter 12.pdf"),
    PyPDFLoader("11Physics/Chapter 13.pdf"),
    PyPDFLoader("11Physics/Chapter 14.pdf"),
    PyPDFLoader("11Physics/Solutions 1.pdf"),
    PyPDFLoader("11Physics/Solutions 2.pdf"),

    PyPDFLoader("12Physics/Chapter 1.pdf"),
    PyPDFLoader("12Physics/Chapter 2.pdf"),
    PyPDFLoader("12Physics/Chapter 3.pdf"),
    PyPDFLoader("12Physics/Chapter 4.pdf"),
    PyPDFLoader("12Physics/Chapter 5.pdf"),
    PyPDFLoader("12Physics/Chapter 6.pdf"),
    PyPDFLoader("12Physics/Chapter 7.pdf"),
    PyPDFLoader("12Physics/Chapter 8.pdf"),
    PyPDFLoader("12Physics/Chapter 9.pdf"),
    PyPDFLoader("12Physics/Chapter 10.pdf"),
    PyPDFLoader("12Physics/Chapter 11.pdf"),
    PyPDFLoader("12Physics/Chapter 12.pdf"),
    PyPDFLoader("12Physics/Chapter 13.pdf"),
    PyPDFLoader("12Physics/Chapter 14.pdf"),
    PyPDFLoader("12Physics/Solutions.pdf"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

docs = text_splitter.split_documents(docs)
print(len(docs))

from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Store the embeddings in VectorDB
from langchain.vectorstores import Chroma

# We save this vector store in a persistent directory so that we can use it in future.
persist_directory = 'docs/chroma/'

# Create the vector store
VectorDB = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

print(VectorDB._collection.count())

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed <br>
**Retrieval section in medium article above** <br>

# Retrieval from VectorDB (Chroma DB)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

persist_directory = 'docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template) # Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    # return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
from langchain.agents import Tool
retrieval_tool = Tool(
    name='RetrievalQA',
    description = 'This tool can be used to get context from the knowledge base of textbooks.',
    func=qa_chain.run, handle_parsing_errors=True
)

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
tools = load_tools(['llm-math'], llm=llm)
tools.append(retrieval_tool)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)
# agent.run("Given equations for variable x, y, and z: x = y + 5, y = z - 3,  z = x * y. Solve for x, y, z.")
agent.run("Use 'retrieval_tool' to answer the following - The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale?")



> Entering new AgentExecutor chain...
I need to find the conversion formula for Fahrenheit to Celsius.
Action: RetrievalQA
Action Input: "Fahrenheit to Celsius conversion formula"
Observation: The Fahrenheit to Celsius conversion formula is: C = (F-32) x 5/9. Thanks for asking!
Thought:Now I need to input the values into the formula.
Action: Calculator
Action Input: (200-32) x 5/9
Observation: Answer: 93.33333333333333
Thought:I now know the final answer.
Final Answer: The reading on the Celsius scale would be 93.33 °C.

> Finished chain.


'The reading on the Celsius scale would be 93.33 °C.'

In [ ]:
question = 'Use retrieval_tool to answer the following, also tell which document help to provide the context - The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale?'
result = qa_chain({"query": question})
result

{'query': 'Use retrieval_tool to answer the following, also tell which document help to provide the context - The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale?',
 'result': 'The temperatures on the Fahrenheit scale are represented between 32° to 212°. The Celsius scale is calibrated from 0° to 100°, where these two temperatures are the freezing point and the boiling point of water, respectively. The temperatures on two scales are related to each other by the following relationship: °C = (°F - 32) × 5/9. Therefore, the temperature on the Celsius scale would be 93.3 °C if the measured temperature on the Fahrenheit scale is 200 °F. Thanks for asking! \n\nContext document: 1.4 Uncertainty in measurement',
 'source_documents': [Document(page_content='10\nchemistry\nIn the laboratory, the volume of liquids \nor solutions can be measured by graduated \ncylinder, burette, pipette, etc. A volumetric flask  is used to prepare a known volu

**Chain-of-Thought and Self-Correction** <br>
1) https://www.kdnuggets.com/some-kick-ass-prompt-engineering-techniques-to-boost-our-llm-models#:~:text=Prompt%20engineering%20has%20emerged%20as,saving%20both%20time%20and%20resources.
2) https://medium.com/@johannes.koeppern/self-consistency-with-chain-of-thought-cot-sc-2f7a1ea9f941
3) https://www.youtube.com/watch?v=SvbK-zO9nzQ
4) https://www.youtube.com/watch?v=gnHOqjfOAsQ

**Gemini-Pro-AI API**:
1) https://www.youtube.com/watch?v=NHt_Th2rgvQ <br>
2) https://www.youtube.com/watch?v=f6ZmtKvPni4 <br>
3) https://www.youtube.com/watch?v=ZrH3yW2beVs <br>
4) https://www.youtube.com/watch?v=G3-YOEVg-xc <br>
5) https://python.langchain.com/docs/integrations/chat/google_generative_ai <br>
6) https://www.analyticsvidhya.com/blog/2023/12/google-gemini-api/#:~:text=Langchain%20and%20Gemini%20Integration,-With%20the%20release&text=First%2C%20we%20create%20the%20llm,function%20will%20generate%20the%20response. <br>

# Working with Gemini Pro API

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from sec import GeminiKey

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GeminiKey)
# result = llm.invoke("Equations for x, y, and z are given: x = y + 5, y = z - 3,  z = x * y. Solve for x, y, z. Return only final values of x, y, z and not the steps using which you reached there.")
result = llm.invoke("The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale? Return only final decimal value upto 2 decimal places and not the steps using which you reached there.")
print(result.content)

93.33


In [ ]:
from langchain.prompts import ChatPromptTemplate
# template = '''Answer the question as based only on the following context:
# {context}
# Question: {question}
# '''

template = '''Answer the question use the following context if needed:
{context}
Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Defining vectordb
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

document_content_description = "Brief summary of a chemistry text"
from langchain.retrievers.self_query.base import SelfQueryRetriever
retriever = SelfQueryRetriever.from_llm(
    llm, vectordb, document_content_description, metadata_field_info =[], verbose=True
)

# output parser
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

from langchain.schema.runnable import RunnableMap
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | llm | output_parser

result = chain.invoke({"question": "The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale? Return only final decimal value and not the steps using which you reached there. "})
# result = chain.invoke({"question": "The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale? "})
print(result)
# print(result.content) -> This is not needed when you use output_parser

93.3


In [ ]:
results = llm.batch(
    [
        "What's 2+2?",
        "What's 3+5?",
    ]
)
for res in results:
    print(res.content)

4
8


# Final OpenAI API Pipeline

In [ ]:
import langchain
import openai

# from langchain_google_genai import ChatGoogleGenerativeAI
# from sec import GeminiKey
# llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GeminiKey)

from sec import OpenAIkey, WolframAlphaKey
key = OpenAIkey

from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
llm = AzureChatOpenAI(azure_endpoint= 'https://amazebotopenai.openai.azure.com/', api_key = key, deployment_name= 'Chat', api_version='2023-05-15')

In [ ]:
query_template = '''Solve this question:
{Question}\n and return the correct option number\n
(1) {Option_1}\n
(2) {Option_2}\n
(3) {Option_3}\n
(4) {Option_4}\n
Output the correct option number (1, 2, 3, or 4). I want the output to not exceed length 1.
A quick tip: Use wolfram-alpha tool provided to solve math questions. This tool is good for math.
Use retrieval_tool to extract context from knowledge base.
Let's think step-by-step to reach the final answer.'''

query_template = PromptTemplate(
    input_variables=["Question", "Option_1", "Option_2", "Option_3", "Option_4"],
    template=query_template
)

chain = LLMChain(llm=llm, prompt=query_template, output_key="correct_option")

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.pal_chain import PALChain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import os
os.environ["WOLFRAM_ALPHA_APPID"] = WolframAlphaKey

python_repl = PythonREPL()
pal_chain = PALChain.from_math_prompt(llm=llm)
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

persist_directory = 'docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
)

python_repl = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run, handle_parsing_errors=True
)

pal_math = Tool(
    name="pal_math",
    description="A LLM Chain which helps to solve word problems.",
    func=pal_chain.run, handle_parsing_errors=True
)

retrieval_tool = Tool(
    name='RetrievalQA',
    description = 'This tool can be used to get context from the knowledge base of textbooks.',
    func=qa_chain.run, handle_parsing_errors=True
)

tools = load_tools(['llm-math', 'wolfram-alpha'], llm=llm)
tools.append(python_repl)
tools.append(pal_math)
tools.append(retrieval_tool)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True)
# agent.run("Given equations for variable x, y, and z: x = y + 5, y = z - 3,  z = x * y. Solve for x, y, z.")
# agent({question:"Use 'retrieval_tool' to answer the following - The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale?"})
# agent.run("Use 'retrieval_tool' to answer the following - The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale?")

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd
from pylatexenc.latex2text import LatexNodes2Text
data = pd.read_csv('2013_Paper1 - MCQS.csv')
data = data.head(5)

def latex_to_text(i):
    data.loc[i, 'Question'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Question'])
    data.loc[i, 'Option1'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option1'])
    data.loc[i, 'Option2'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option2'])
    data.loc[i, 'Option3'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option3'])
    data.loc[i, 'Option4'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option4'])

def predict(i):
    try:
        query_template.format(Option_1=data.loc[i, 'Option1'], Option_2=data.loc[i, 'Option2'], Option_3=data.loc[i, 'Option3'], Option_4=data.loc[i, 'Option4'], Question=data.loc[i, 'Question'])
        result = agent.invoke(query_template)
    except:
        return 'Error'
    else:
        return result['output']


In [ ]:
score = 0 # 1 for each correct answer
for i in range(1):
    latex_to_text(i)
    pred = predict(i)
    print(pred)
    while(pred == 'Error' or len(pred) > 1):
        pred = predict(i)
        print(pred)
    pred = int(pred)
    print(pred)
    if pred == 1:
        if data['Answer1'][i] == 1:
            score+=1
    elif pred == 2:
        if data['Answer2'][i] == 1:
            score+=1
    elif pred == 3:
        if data['Answer3'][i] == 1:
            score+=1
    elif pred == 4:
        if data['Answer4'][i] == 1:
            score+=1
print(score)


Agent stopped due to iteration limit or time limit.
(1)
1
1
0


In [ ]:
score = 0 # 1 for each correct answer
for i in range(1): # 1 should be replaced with len(data)
    latex_to_text(i)
    while(1):
        pred = predict(i)
        if pred == 'Error' or len(pred) > 1:
            print(pred)
            pred = predict(i)
        else:
            break
    pred = int(pred)
    print(pred)
    if pred == 1:
        if data['Answer1'][i] == 1:
            score+=1
    elif pred == 2:
        if data['Answer2'][i] == 1:
            score+=1
    elif pred == 3:
        if data['Answer3'][i] == 1:
            score+=1
    elif pred == 4:
        if data['Answer4'][i] == 1:
            score+=1
print(score)

Python REPL can execute arbitrary code. Use with caution.
Python REPL can execute arbitrary code. Use with caution.


1:15
Error
Option B (2)
Error
Agent stopped due to iteration limit or time limit.
12
Error
2
2
0


In [ ]:
# Implementing self-consistency with CoT
from statistics import mode

def evaluate(pred, score, i):
    if pred == 1:
        if data['Answer1'][i] == 1:
            score+=1
    elif pred == 2:
        if data['Answer2'][i] == 1:
            score+=1
    elif pred == 3:
        if data['Answer3'][i] == 1:
            score+=1
    elif pred == 4:
        if data['Answer4'][i] == 1:
            score+=1
    return score

def solve(i):
    pred = predict(i)
    print(pred)
    while(pred == 'Error' or len(pred) > 1):
        pred = predict(i)
        print(pred)
    pred = int(pred)
    print(pred)
    return pred

score = 0
for i in range(1): # 1 should be replaced with len(data)
    latex_to_text(i)
    responses = []
    for iter in range(4):
        responses.append(solve(i))
    final_pred = mode(responses)
    score = evaluate(final_pred, score, i)
print(score)

# In the output below we can see that there are many time we see some Errors. Such errors are ignored and the llm is called again to get the actual answer.
# The output as '3' or '2' is the option number selected. Output as 'Option 2' or 'Option_3' or '(3) Option_3' is ignored.
# We see the responses we get are [3,3,3,2]. Mode=3 which is the final answer.
# Self-Consistency helps to select the correct option.

Error
The value of x in the equation 2x + 5 = 13 is (1) 4.
3
3
Error
Option 2
Error


Python REPL can execute arbitrary code. Use with caution.


40
Option_3
Error
N/A
Error
Error
Error
20.0
Error
(3) Option_3
Agent stopped due to iteration limit or time limit.
Error
Error
Agent stopped due to iteration limit or time limit.
3
3
Agent stopped due to iteration limit or time limit.
3
3
The probability of rolling a 6 on a fair dice is 1/6.
2
2
1


# Final GeminiPro API Pipeline

In [ ]:
import langchain
import openai

from sec import OpenAIkey, WolframAlphaKey
key = OpenAIkey

from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain_google_genai import ChatGoogleGenerativeAI
from sec import GeminiKey
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GeminiKey)

In [ ]:
query_template = '''Solve this question:
{Question}\n and return the correct option number\n
(1) {Option_1}\n
(2) {Option_2}\n
(3) {Option_3}\n
(4) {Option_4}\n
Output the correct option number (1, 2, 3, or 4). I want the output to not exceed length 1.
A quick tip: Use wolfram-alpha tool provided to solve math questions. This tool is good for math.
Use retrieval_tool to extract context from knowledge base.
Let's think step-by-step to reach the final answer.'''

query_template = PromptTemplate(
    input_variables=["Question", "Option_1", "Option_2", "Option_3", "Option_4"],
    template=query_template
)

chain = LLMChain(llm=llm, prompt=query_template, output_key="correct_option")

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.pal_chain import PALChain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import os
os.environ["WOLFRAM_ALPHA_APPID"] = WolframAlphaKey

python_repl = PythonREPL()
pal_chain = PALChain.from_math_prompt(llm=llm)
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

persist_directory = 'docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
)

python_repl = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run, handle_parsing_errors=True
)

pal_math = Tool(
    name="pal_math",
    description="A LLM Chain which helps to solve word problems.",
    func=pal_chain.run, handle_parsing_errors=True
)

retrieval_tool = Tool(
    name='RetrievalQA',
    description = 'This tool can be used to get context from the knowledge base of textbooks.',
    func=qa_chain.run, handle_parsing_errors=True
)

tools = load_tools(['llm-math', 'wolfram-alpha'], llm=llm)
tools.append(python_repl)
tools.append(pal_math)
tools.append(retrieval_tool)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True)

In [ ]:
import pandas as pd
from pylatexenc.latex2text import LatexNodes2Text
data = pd.read_csv('2013_Paper1 - MCQS.csv')
data = data.head(5)

def latex_to_text(i):
    data.loc[i, 'Question'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Question'])
    data.loc[i, 'Option1'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option1'])
    data.loc[i, 'Option2'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option2'])
    data.loc[i, 'Option3'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option3'])
    data.loc[i, 'Option4'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option4'])

def predict(i):
    try:
        query_template.format(Option_1=data.loc[i, 'Option1'], Option_2=data.loc[i, 'Option2'], Option_3=data.loc[i, 'Option3'], Option_4=data.loc[i, 'Option4'], Question=data.loc[i, 'Question'])
        result = agent.invoke(query_template)
    except:
        return 'Error'
    else:
        return result['output']


In [ ]:
# Implementing self-consistency with CoT
from statistics import mode

def evaluate(pred, score, i):
    if pred == 1:
        if data['Answer1'][i] == 1:
            score+=1
    elif pred == 2:
        if data['Answer2'][i] == 1:
            score+=1
    elif pred == 3:
        if data['Answer3'][i] == 1:
            score+=1
    elif pred == 4:
        if data['Answer4'][i] == 1:
            score+=1
    return score

def solve(i):
    pred = predict(i)
    print(pred)
    while(pred == 'Error' or len(pred) > 1):
        pred = predict(i)
        print(pred)
    pred = int(pred)
    print(pred)
    return pred

score = 0
for i in range(1): # 1 should be replaced with len(data)
    latex_to_text(i)
    responses = []
    for iter in range(4):
        responses.append(solve(i))
    final_pred = mode(responses)
    score = evaluate(final_pred, score, i)
print(score)



Agent stopped due to iteration limit or time limit.
Error
625/4
2
2
4
4
Agent stopped due to iteration limit or time limit.
Error
null
Error
2
2
Error
Error
Agent stopped due to iteration limit or time limit.
Agent stopped due to iteration limit or time limit.
4
4
0


### TESTING THE PIPELINE WITH A DATASET OF QUESTIONS. PIPELINE HAS SOME CHANGES w.r.t. the format of result

In [ ]:
# Install important libraries
!pip install langchain
!pip install openai
!pip install pylatexenc
!pip install langchain_experimental
!pip install sentence_transformers
!pip install chromadb
!pip install wolframalpha
from google.colab import userdata
OpenAIkey = userdata.get('OpenAIkey')
WolframAlphaKey = userdata.get('WolframAlphaKey')

In [ ]:
import langchain
import openai

# from langchain_google_genai import ChatGoogleGenerativeAI
# from sec import GeminiKey
# llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GeminiKey)

key = OpenAIkey

from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
llm = AzureChatOpenAI(azure_endpoint= 'https://amazebotopenai.openai.azure.com/', api_key = key, deployment_name= 'Chat', api_version='2023-05-15')

In [ ]:
query_template = '''Solve this question:
{Question}\n and return the correct option number\n
(1) {Option_1}\n
(2) {Option_2}\n
(3) {Option_3}\n
(4) {Option_4}\n
Output the correct option number (1, 2, 3, or 4). I want the output to not exceed length 1.
A quick tip: Use wolfram-alpha tool provided to solve math questions. This tool is good for math.
Use retrieval_tool to extract context from knowledge base.
Let's think step-by-step to reach the final answer.'''

query_template = PromptTemplate(
    input_variables=["Question", "Option_1", "Option_2", "Option_3", "Option_4"],
    template=query_template
)

chain = LLMChain(llm=llm, prompt=query_template, output_key="correct_option")

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.pal_chain import PALChain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from google.colab import drive
drive.mount('/content/drive')

import os
os.environ["WOLFRAM_ALPHA_APPID"] = WolframAlphaKey

python_repl = PythonREPL()
pal_chain = PALChain.from_math_prompt(llm=llm)
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

persist_directory = '/content/drive/My Drive/Colab Notebooks/Langchain/docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
)

python_repl = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run, handle_parsing_errors=True
)

pal_math = Tool(
    name="pal_math",
    description="A LLM Chain which helps to solve word problems.",
    func=pal_chain.run, handle_parsing_errors=True
)

retrieval_tool = Tool(
    name='RetrievalQA',
    description = 'This tool can be used to get context from the knowledge base of textbooks.',
    func=qa_chain.run, handle_parsing_errors=True
)

tools = load_tools(['llm-math', 'wolfram-alpha'], llm=llm)
tools.append(python_repl)
tools.append(pal_math)
tools.append(retrieval_tool)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True)
# agent.run("Given equations for variable x, y, and z: x = y + 5, y = z - 3,  z = x * y. Solve for x, y, z.")
# agent({question:"Use 'retrieval_tool' to answer the following - The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale?"})
# agent.run("Use 'retrieval_tool' to answer the following - The measured temperature on the Fahrenheit scale is 200 °F. What would the reading be on the Celsius scale?")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from pylatexenc.latex2text import LatexNodes2Text
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Langchain/2013_Paper1 - MCQS.csv')

def latex_to_text(i):
    data.loc[i, 'Question'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Question'])
    data.loc[i, 'Option1'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option1'])
    data.loc[i, 'Option2'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option2'])
    data.loc[i, 'Option3'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option3'])
    data.loc[i, 'Option4'] = LatexNodes2Text().latex_to_text(data.loc[i, 'Option4'])

def predict(i):
    try:
        query_template.format(Option_1=data.loc[i, 'Option1'], Option_2=data.loc[i, 'Option2'], Option_3=data.loc[i, 'Option3'], Option_4=data.loc[i, 'Option4'], Question=data.loc[i, 'Question'])
        result = agent.invoke(query_template)
    except:
        return 'Error'
    else:
        return result['output']


In [ ]:
# Implementing self-consistency with CoT
from statistics import mode

def evaluate(pred, score, i):
    if pred == 1:
        if data['Answer1'][i] == 1:
            score+=1
    elif pred == 2:
        if data['Answer2'][i] == 1:
            score+=1
    elif pred == 3:
        if data['Answer3'][i] == 1:
            score+=1
    elif pred == 4:
        if data['Answer4'][i] == 1:
            score+=1
    return score

def solve(i):
    pred = predict(i)
    while(pred == 'Error' or len(pred) > 1):
        pred = predict(i)
    pred = int(pred)
    return pred

def solve_section(start_row, last_row):
    # start_row denotes the row number where the questions for that particular section starts.
    # num_of_questions denote the number of questions in the section.
    score = 0
    i = start_row
    while(i <= last_row):
        latex_to_text(i)
        responses = []
        for iter in range(4):
            responses.append(solve(i))
        final_pred = mode(responses)
        score += evaluate(final_pred, score, i)
        print(score)
        i += 1
    return score

# In the output below we can see that there are many time we see some Errors. Such errors are ignored and the llm is called again to get the actual answer.
# The output as '3' or '2' is the option number selected. Output as 'Option 2' or 'Option_3' or '(3) Option_3' is ignored.
# We see the responses we get are [3,3,3,2]. Mode=3 which is the final answer.
# Self-Consistency helps to select the correct option.

In [ ]:
def find_end_section(section, start_row, last_row_year):
    # len(data)-1 is the total number of questions
    while(start_row < last_row_year and data['Subject'][start_row] == section):
        start_row+=1
    return start_row-1

def find_end_year(year, start_row):
    while(start_row < len(data) and data['Year'][start_row] == year):
        start_row+=1
    return start_row-1

result= []

# Solving an year
start_row_year = 0
while(start_row_year < len(data)):
    year = data['Year'][start_row_year]
    last_row_year = find_end_year(year, start_row_year)
    # print(start_row_year, last_row_year)
    # Solving a section
    start_row_section = 0
    while(start_row_section < last_row_year):
        section=data['Subject'][start_row_section]
        last_row_section = find_end_section(section, start_row_section, last_row_year)
        print(start_row_section, last_row_section)
        score = solve_section(start_row_section, last_row_section)

        if(section=='Physics'):
            physics_score = score
        elif(section == 'Maths'):
            math_score = score
        elif(section == 'Chemistry'):
            chemistry_score = score
        print(f'Completed {section} section of year {year}')
        start_row_section = last_row_section+1 # Moving to next section

    total_num_of_questions = last_row_year-start_row_year+1
    # Adding result for an year
    result.append(
        {
        'Year': year,
        'Total Questions': total_num_of_questions,
        'Correct Questions': score,
        'Incorrect Questions': total_num_of_questions-score,
        'Physics': physics_score,
        'Chemistry': chemistry_score,
        'Math': math_score,
        'Marks Obtained': score*4
        }
    )

    result = pd.DataFrame(result)
    print(result)
    result.to_csv('Result.csv', index=False)
    start_row_year = last_row_year+1 # Moving to next Year questions